[**Открыть тетрадку в colab**](https://colab.research.google.com/github/m12sl/voronovo-2019/blob/master/cifar10_tutorial.ipynb)

In [ ]:
%matplotlib inline

[Перевод туториала](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) с небольшими добавлениями.

В этой тетрадке мы натренируем небольшую сверточную сеть для классификации картинок из датасета Cifar-10.
Цель познакомить с основными шагами тренировки:

1. Описать генератор данных
2. Написать нейронную сеть
3. Задать функцию ошибки
4. Написать тренировочный цикл
5. Натренировать модель
6. Попробовать применить.


В данном случае код для загрузки данных на train/validation уже написан за нас и находится в `torchvision.datasets`.

Обычно нам требуется три компоненты:

1. **Dataset** наследуется от `torch.utils.data.Dataset`.

    В классе должно быть реализовано два метода:

    - `__getitem__(self, item)`, он возвращает семпл (в данном случае картинку и номер класса для нее)
    - `__len__(self)`, количество примеров в датасете.

2. **DataLoader** часто используется готовый `torch.utils.data.DataLoader`.

    Принимает на вход `dataset`, размер батча и дополнительные параметры.
    Внутри вызывает `dataset.__getitem__` нужное количество раз и складывает семплы в батч.

3. **transform** -- функция для предобработки картинок, обычно подается в Dataset при инициализации. Полезна для аугментаций и получения более подходящих представлений.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Cifar10

Состоит из цветных картинок 32х32, содержит 10 классов:
‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’,
‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

Картинки в нем по умолчанию представлены в виде массивов пикселей со значениями от 0 до 1. При загрузке мы перевели их в интервал $(-1, 1)$

Let us show some of the training images, for fun.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

## Пишем сверточную сеть.

В pytorch принят **channels-first** порядок осей. 
Это означает, что в сеть приходят тензоры с размерами `[batch_size, channels, width, height]`. 


Сверточные сети обычно собираются из последовательности слоев:

### Convolution
https://pytorch.org/docs/stable/nn.html#convolution-layers

По тензору бежит скользящее окно и в нем вычисляется свертка с ядром.
Обычно говорят о пространственных размерах сверток, например 1x1 или 3x3  свертки, подразумевая, что ядра имеют размер `[1,1,ch]` или `[3,3,ch]`.

Сейчас часто используются чуть более сложные варианты сверток: 
- dilated (atrous, дырявые), 
- depth-wise
- pointwise
- separable
- group


### Pooling
https://pytorch.org/docs/stable/nn.html#pooling-layers

Действуют аналогично свертках, но не имеют весов, а в бегущем окне вычисляется какая-нибудь функция, например max или mean.


### Global pooling (Adaptive Pooling)
https://pytorch.org/docs/stable/nn.html#adaptivemaxpool1d

Глобальные пулинги (в pytorch они называются adaptive) убирают пространственные размерности, превращая `[bs, ch, h, w]` в `[bs, ch, 1, 1]`.



Удобно выделять в сверточных сетях две части: полносверточную (body, feature extractor, тушка) и классификатор (head, голова).

Классификатор обычно состоит из полносвязных слоев (и где-то может обозначаться как MLP, MLP-head), и требует фиксированного размера тензоров (batch_size может варьироваться, но остальные размерности фиксированы).

Полносверточная часть обычно может работать на входах произвольных размеров (не меньше минимального).


Чтобы объединить эти две части используют какую-нибудь из операций: **Flatten** или **Global Pooling**.

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.body = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
        )
        self.head = nn.Sequential(
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
        )
        

    def forward(self, x):
        x = self.body(x)
        x = x.view(x.size(0), -1) # flatten
        x = self.head(x)
        return x


net = Net()

# check network consistency
print(net(torch.zeros([32, 3, 32, 32])).size())

## Функция ошибки и оптимизатор

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  # объединяет LogSoftmax и NLL
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
! pip install torchviz

In [ ]:
# нарисуем вычислительный граф для бекпропа
from torchviz import make_dot, make_dot_from_trace

out = net(images)
loss = criterion(net(images), labels)
make_dot(loss, params=dict(net.named_parameters()))

Для отображения прогресса нам потребуются дополнительные функции

In [ ]:
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
from IPython.display import clear_output

# Some auxilary function for plots
def plot_history(log, name=None):
    """log is list of dictionaries like 
        [
            {'train_step': 0, 'train_loss': 10.0, 'train_acc': 0.0}, 
            ...
            {'train_step': 100, 'val_loss': 0.1, 'val_acc': 0.9},
            ...
        ]
    """
    if name is None:
        name='loss'
    train_points, val_points = [], []
    train_key = 'train_{}'.format(name)
    val_key = 'val_{}'.format(name)

    for entry in log:
        if train_key in entry:
            train_points.append((entry['train_step'], entry[train_key]))
        if val_key in entry:
            val_points.append((entry['train_step'], entry[val_key]))
    
    plt.figure()
    plt.title(name)
    x, y = list(zip(*train_points))
    plt.plot(x, y, label='train', zorder=1)
    x, y = list(zip(*val_points))
    plt.scatter(x, y, label='val', zorder=2, marker='+', s=180, c='orange')
    
    plt.legend(loc='best')
    plt.grid()
    plt.show()

## Тренировочный цикл 

Эпоха -- один проход по датасету.
После тренировочной эпохи будем подсчитывать метрики на тесте.

In [ ]:
def train_model(model, optimizer, criterion, train_loader, val_loader, batch_size=32, epochs=10, device=None):
    log = []
    train_step = 0
    model = model.to(device)
    for epoch in range(epochs):
        model.train()
        for x, y in tqdm(train_loader):
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = model(x)

            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(output, 1)
            acc = (predicted == y).sum().item() / x.size(0)
            
            log.append(dict(
                train_loss=loss.item(),
                train_acc=acc,
                train_step=train_step,
            ))
            train_step += 1

        # валидационные метрики надо усредних за все валидационные батчи
        # hint: для аккумулирования величин удобно взять defaultdict
        tmp = defaultdict(list)
        model.eval()
        for x, y in tqdm(val_loader):
            x = x.to(device)
            y = y.to(device)
            with torch.no_grad():
                # <your code here>
                output = model(x)
                loss = criterion(output, y)
                _, predicted = torch.max(output, 1)
                acc = (predicted == y).data.numpy()
                tmp['acc'].append(acc)
                tmp['loss'].append(loss.item())
                
                
        log.append(dict(
            val_loss = np.mean(tmp['loss']),  # скаляры
            val_acc = np.concatenate(tmp['acc']).mean(),  # массивы, возможно разной длины
            train_step=train_step,
        ))
        
        clear_output()
        plot_history(log, name='loss')
        plot_history(log, name='acc')

In [ ]:
# инициализируем заново
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = Net()
optimizer = optim.Adam(net.parameters(), lr=1e-2)

train_model(net, optimizer, nn.CrossEntropyLoss(), trainloader, testloader, epochs=10, device=device)

## Применим обученную сеть к тренировочным данным

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# make predictions:
outputs = net(images)
_, predictions = outputs.topk(1, dim=-1)
predictions = predictions.cpu().numpy().flatten()
predicted_classes = [classes[i] for i in predictions[:4]]

# print images
imshow(torchvision.utils.make_grid(images[:4, ...]))

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
print('Predicted: ', ' '.join(predicted_classes))